In [ ]:
import kagglehub
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

import warnings
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

Описание dataset: Контекст задачи Записи о продажах с 2011 по 2014 годы, включающие 3 основные категории товаров и 17 подкатегорий, представлены с детализацией по регионам и сегментам.

Описание ключевых признаков: State (Штат): Штат, в котором проживает клиент (внутри страны).

City (Город): Город проживания клиента.

Region (Регион): Географический регион (например, Запад, Юг и т.д.).

Segment (Сегмент): Тип клиента/рынка: Consumer, Corporate, Home Office и др.

Ship Mode (Способ доставки): Метод доставки заказа до клиента.

Category (Категория): Основная категория товара (например, Мебель, Техника).

Sub-Category (Подкатегория): Более точная классификация товаров.

Product Name (Наименование товара): Название конкретного продукта.

Discount (Скидка): Скидка, применённая к товару.

Sales (Продажи): Общая сумма продаж по заказу.

Profit (Прибыль): Полученная прибыль по заказу.

Quantity (Количество): Количество единиц товара в заказе.

Feedback (Отзыв): Признак — оставил ли клиент отзыв (True/False).

In [ ]:
print("Скачиваем датасет через kagglehub...")
path = kagglehub.dataset_download("braniac2000/retail-dataset")
print(f"Путь к скачанной папке: {path}")

xlsx_path = None
for root, dirs, files in os.walk(path):
    for f in files:
        if f.endswith(".xlsx"):
            xlsx_path = os.path.join(root, f)
            break
    if xlsx_path:
        break

if xlsx_path is None:
    raise ValueError("Файл .xlsx не найден!")

print(f"Используем Excel-файл: {xlsx_path}")

df = pd.read_excel(xlsx_path, sheet_name='Order Data')
print(f"Исходный размер данных: {df.shape}")


Скачивание файла dataset, который проверяет установился ли файл, либо же нет.

In [ ]:
# Отфильтровать строки без пропусков (NaN)
df_no_nan = df.dropna()

# Получить описательную статистику по отфильтрованным данным
desc_no_nan = df_no_nan.describe(include='all')

# Показать результат
num_non_null_rows = df.dropna().shape[0]

print(f"Количество строк без пропусков: {num_non_null_rows}")


In [ ]:
df.head()
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

def analyze_profit_by_category(df, col, top_n=10):
    # Считаем суммарную прибыль по всем уникальным значениям (включая отрицательные)
    total_profit = df.groupby(col)['Profit'].sum()

    # Отбираем топ-N значений с наибольшей суммарной прибылью (по убыванию)
    top_vals = total_profit.sort_values(ascending=False).head(top_n).index.tolist()

    # Фильтруем исходный датафрейм по топ-N
    df_top = df[df[col].isin(top_vals)].copy()

    # Считаем метрики по всем заказам (включая отрицательные)
    profit_sum = df_top.groupby(col)['Profit'].sum().reindex(top_vals)
    avg_profit_all = df_top.groupby(col)['Profit'].mean().reindex(top_vals)

    # Средняя прибыль только по прибыльным заказам (для информации)
    avg_profit_positive = df_top[df_top['Profit'] > 0].groupby(col)['Profit'].mean().reindex(top_vals)

    order_count = df_top.groupby(col).size().reindex(top_vals)

    # Процент убыточных заказов
    loss_counts = df_top[df_top['Profit'] < 0].groupby(col).size().reindex(top_vals).fillna(0)
    loss_percent = (loss_counts / order_count * 100).fillna(0)

    # Цвета для топовых значений
    palette = sns.color_palette("tab10", n_colors=len(top_vals))
    color_dict = dict(zip(top_vals, palette))

    fig, axes = plt.subplots(1, 3, figsize=(20, 6))

    # Суммарная прибыль
    sns.barplot(x=profit_sum.index, y=profit_sum.values, palette=[color_dict[x] for x in profit_sum.index], ax=axes[0])
    axes[0].set_title(f"Суммарная прибыль по {col}")
    axes[0].set_xlabel(col)
    axes[0].set_ylabel("Суммарная прибыль")
    axes[0].tick_params(axis='x', rotation=45)

    # Средняя прибыль (все заказы)
    sns.barplot(x=avg_profit_all.index, y=avg_profit_all.values, palette=[color_dict[x] for x in avg_profit_all.index], ax=axes[1])
    axes[1].set_title(f"Средняя прибыль на заказ по {col} (все заказы)")
    axes[1].set_xlabel(col)
    axes[1].set_ylabel("Средняя прибыль")
    axes[1].tick_params(axis='x', rotation=45)

    # Количество заказов
    sns.barplot(x=order_count.index, y=order_count.values, palette=[color_dict[x] for x in order_count.index], ax=axes[2])
    axes[2].set_title(f"Количество заказов по {col}")
    axes[2].set_xlabel(col)
    axes[2].set_ylabel("Количество заказов")
    axes[2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

    # Распределение прибыли по категориям
    print(f"\n📊 Распределение прибыли по категориям внутри топ-{top_n} {col}")
    profit_by_cat = df_top.groupby([col, 'Category'])['Profit'].sum().unstack(fill_value=0).reindex(top_vals)

    profit_by_cat.plot(kind='bar', stacked=True, figsize=(16, 7), colormap='tab20')
    plt.title(f"Распределение прибыли по категориям в топ-{top_n} {col}")
    plt.xlabel(col)
    plt.ylabel("Суммарная прибыль")
    plt.xticks(rotation=45)
    plt.legend(title='Категории', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    best_sum_profit = profit_sum.idxmax()
    worst_sum_profit = profit_sum.idxmin()

    def fmt_int(n):
        return f"{int(n):,}".replace(",", " ")

    def fmt_float(n):
        return f"{n:.2f}"

    print(f"\n💡 Вывод по '{col}':")
    print(f"- Максимальная суммарная прибыль у '{best_sum_profit}' — {fmt_int(profit_sum.max())}")
    print(f"- Минимальная суммарная прибыль у '{worst_sum_profit}' — {fmt_int(profit_sum.min())}")
    print(f"- Средняя прибыль на заказ (все заказы) варьируется от {fmt_float(avg_profit_all.min())} до {fmt_float(avg_profit_all.max())}")
    print(f"- Средняя прибыль на заказ (только прибыльные) варьируется от {fmt_float(avg_profit_positive.min())} до {fmt_float(avg_profit_positive.max())}")
    print(f"- Количество заказов в топ-{top_n} '{col}' варьируется от {fmt_int(order_count.min())} до {fmt_int(order_count.max())}")
    print(f"- Процент убыточных заказов в топ-{top_n} '{col}':")
    for val in top_vals:
        print(f"  • {val}: {fmt_float(loss_percent[val])}%")


analyze_profit_by_category(df, 'Customer Name')



Максимальная прибыль концентрируется у небольшого числа клиентов с большим числом заказов и высокой средней прибылью. При этом у некоторых клиентов высокий процент убыточных заказов, что может на акции или скидки, либо на процент возвратов.

In [ ]:
analyze_profit_by_category(df, 'Country')

Как видно, основная прибыль поступает из крупных городов с развитой экономикой. При этом спрос разделён не однородно, так многие страны имели низкую доходность, но скорее вызвано тем, что часть товаров не было, а с другой стороны в более развитых странах наблюдается больших процент убыточных заказов



In [ ]:
analyze_profit_by_category(df, 'State')

Максимальная прибыль и количество заказов сосредоточены в экономически развитых штатах (например, England). Высокий процент убыточных заказов в некоторых штатах указывает на снижение маржинальности, возможно из-за скидок или возвратов.

In [ ]:
analyze_profit_by_category(df, 'City')

Как видно, основные заказы поступают из городов с высокой экономической активностью. При этом спрос по категориям товаров варьируется: в некоторых городах преобладает покупка технологичных товаров, тогда как в других спрос на другие товары.

In [ ]:
analyze_profit_by_category(df, 'Region')

Центральные районы приносят максимальную прибыль при умеренном уровне убыточных заказов. Южные показывает минимальную прибыль с похожим уровнем убытков. В Северный высокий процент убыточных заказов снижает эффективность продаж. Разные регионы требуют разных стратегий из-за отличий в прибыльности и рисках, а также то, что центральный регион чаще всего является регионом столицей, из-за чего количество клиентов больше.

In [ ]:
analyze_profit_by_category(df, 'Segment')

Сегмент Consumer (частные покупатели) лидирует по прибыли благодаря большему числу мелких заказов и разнообразию ассортимента, но из‑за скидок и возвратов у них выше риск убыточных сделок. Corporate (корпоративные клиенты) стабилен в объёмах и марже, т.к. работает по договорным ценам. Home Office (домашние офисы) генерирует меньше прибыли из‑за низких заказных объёмов и ограниченного ассортимента. Каждый сегмент требует своей ценовой и ассортиментной стратегии.



In [ ]:
analyze_profit_by_category(df, 'Ship Mode')

Economy справляется с большим количеством дешёвых отправлений в крупных рынках, поэтому её убыточность 22% — много малых заказов и возвратов. Priority (22%) и Economy Plus (~21%) скорее всего какая-та премиум услуга платежеспособных регионах, где клиенты готовы переплачивать за скорость доставки. Сюдя по тому что прочитал.

In [ ]:
analyze_profit_by_category(df, 'Category')

В большинстве своём офисные вещи популярны, т.к. многим офисам нужны столы, стулья и т.д.. Технологии меньше, но дороже, т.к., например, нужны компьютеры для анализа данных. Фурнитура проигрывает по всем фронтам, т.к. она стоит меньше, но всё же заказывается.

In [ ]:
analyze_profit_by_category(df, 'Sub-Category')

Чаще покупателям нужны произведения искуства для украшения комнаты, а также другие товары для обстановки комнат.


In [ ]:
analyze_profit_by_category(df, 'Feedback?')

Чаще люди полагаются на отзывы при заказе товаров, но убыточность одинаково, т.к. неизвестно понравится человеку тот или иной товар.

Feature Engineering. Корреляция новых колонок с таргетом. Feature Importances. Простая модель.

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
import numpy as np

# --- Предобработка ---

df['Feedback?'] = df['Feedback?'].astype(int)
df['Price_per_unit'] = df['Sales'] / df['Quantity']
df['Profit_margin'] = df['Profit'] / df['Sales']
if 'Discount' in df.columns:
    df['Discount_flag'] = (df['Discount'] > 0).astype(int)
else:
    df['Discount_flag'] = 0
df['Has_Feedback'] = df['Feedback?']

cols_to_drop = ['Customer Name', 'Country', 'State', 'City', 'Product Name',
                'Order ID', 'Customer ID', 'Discount', 'Year']
df = df.drop(cols_to_drop, axis=1, errors='ignore')

datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns
df = df.drop(datetime_cols, axis=1)

for col in ['Category', 'Sub-Category', 'Region', 'Segment', 'Ship Mode']:
    if col in df.columns and df[col].dtype == 'object':
        df[col] = LabelEncoder().fit_transform(df[col])

num_cols = df.select_dtypes(include=['number']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

numeric_cols = ['Sales', 'Profit', 'Quantity', 'Price_per_unit', 'Profit_margin', 'Discount_flag', 'Has_Feedback']
numeric_cols = [col for col in numeric_cols if col in df.columns]

corr = df[numeric_cols + ['Profit']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

# --- Модель ---

X = df.drop('Profit', axis=1)
y = (df['Profit'] > 0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

y_pred = (model.predict(X_test) > 0.5).astype(int)
print('Accuracy:', accuracy_score(y_test, y_pred))

# --- График распределения результатов ---

results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})

tp = np.sum((results_df['Actual'] == 1) & (results_df['Predicted'] == 1))
tn = np.sum((results_df['Actual'] == 0) & (results_df['Predicted'] == 0))
fp = np.sum((results_df['Actual'] == 0) & (results_df['Predicted'] == 1))
fn = np.sum((results_df['Actual'] == 1) & (results_df['Predicted'] == 0))

plt.bar(['True Positive', 'True Negative', 'False Positive', 'False Negative'],
        [tp, tn, fp, fn], color=['green', 'green', 'red', 'red'])
plt.title('Распределение предсказаний модели')
plt.ylabel('Количество')
plt.show()


👩‍🎓Эксперименты с моделями машинного обучения/глубокого обучения. По одной из каждого семейства. Линейные, деревья, модификации градиентного бустинга , нейронные сети. На основе результатов выбрать лучшую - делаем кросс-валидацию(не менее 5 фолдов) и итоговый вывод."

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np

sns.set(style="whitegrid")

# === Формируем X и y ===
X = df.drop(['Profit', 'Profit_Positive'], axis=1)
y = df['Profit_Positive']

# === Определяем модели ===
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}

# === Кросс-валидация классических моделей ===
print("Кросс-валидация классических моделей (5 фолдов):\n")
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    cv_results[name] = scores
    print(f"{name}: средняя точность = {scores.mean():.4f}, std = {scores.std():.4f}")

# === Кросс-валидация нейросети вручную ===
def create_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_shape=(input_dim,)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

print("\nНейросеть (ручная кросс-валидация 5 фолдов):")
nn_accuracies = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X), 1):
    X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
    y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]

    model = create_nn_model(X.shape[1])
    model.fit(X_train_cv, y_train_cv, epochs=10, batch_size=32, verbose=0)
    preds = (model.predict(X_val_cv) > 0.5).astype(int)
    acc = accuracy_score(y_val_cv, preds)
    nn_accuracies.append(acc)
    print(f"  Fold {fold}: accuracy = {acc:.4f}")

cv_results['Neural Network'] = np.array(nn_accuracies)
print(f"Neural Network: средняя точность = {np.mean(nn_accuracies):.4f}, std = {np.std(nn_accuracies):.4f}")

# === Итог: выводим лучшую модель ===
best_model = max(cv_results, key=lambda k: cv_results[k].mean())
print(f"\nЛучшая модель: {best_model} с accuracy = {cv_results[best_model].mean():.4f} ± {cv_results[best_model].std():.4f}")

# === График сравнения моделей ===
names = list(cv_results.keys())
means = [cv_results[name].mean() for name in names]
stds = [cv_results[name].std() for name in names]

plt.figure(figsize=(10,6))
plt.bar(names, means, yerr=stds, capsize=5, color='skyblue')
plt.ylabel('Средняя точность (accuracy)')
plt.title('Сравнение моделей по accuracy (5-fold CV)')
plt.ylim(0, 1)
plt.grid(axis='y')
plt.show()


Лучшая модель: Random Forest с accuracy = 1.0000 ± 0.0000 наравне Gradient Boosting